# Proyecto 2

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 1. Seleccionar un video corto
video_path = "path/to/your/video.mp4"

In [ ]:
# 2. Cargar el video utilizando OpenCV
video = cv2.VideoCapture(video_path)

In [ ]:
# 3. Definir una región de interés
_, first_frame = video.read()
bbox = cv2.selectROI("Select Object", first_frame, False)
cv2.destroyAllWindows()

In [ ]:
# 4. Implementar al menos dos algoritmos de seguimiento de objetos
tracker_types = ['MOSSE', 'KCF']
trackers = [cv2.TrackerMOSSE_create(), cv2.TrackerKCF_create()]

# Inicializar los trackers con el primer frame y la región de interés
for tracker in trackers:
    tracker.init(first_frame, bbox)

In [ ]:
# 5. Para cada modelo, mostrar la trayectoria del objeto en el plano XY
centers = {t: [] for t in tracker_types}

while True:
    ret, frame = video.read()

    if not ret:
        break

    for i, tracker in enumerate(trackers):
        ret, bbox = tracker.update(frame)

        if ret:
            x, y, w, h = [int(e) for e in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            center = (x + w // 2, y + h // 2)
            centers[tracker_types[i]].append(center)

            # Dibujar la trayectoria en el video
            for j in range(1, len(centers[tracker_types[i]])):
                cv2.line(frame, centers[tracker_types[i]][j - 1], centers[tracker_types[i]][j], (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Lost", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

    cv2.imshow("Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video.release()
cv2.destroyAllWindows()

In [ ]:
# 5.2. Graficar la trayectoria del objeto en el plano XY
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

for i, t in enumerate(tracker_types):
    xs, ys = zip(*centers[t])
    ax[i].plot(xs, ys, 'r-')
    ax[i].set_title(f"Trajectory using {t} tracker")
    ax[i].set_xlabel("X")
    ax[i].set_ylabel("Y")
    ax[i].invert_yaxis()

plt.show()